# Assignment 3 -- Naive Bayes
## Important: In the part of problem statement I where we were supposed to find conditional probabilities, we did it by manually coding it and by using CountVectorizer(Used the approach of Term Document Matrix) both, so that we could match our results

In [2]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
import pandas as pd
import numpy as np
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to C:\Users\Utkarsh Pratap
[nltk_data]     Singh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Utkarsh Pratap
[nltk_data]     Singh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Problem Statement-1

In [3]:
df=pd.read_csv('traindata.csv') #training_dataset

In [4]:
df.head()

,category,text,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,science,Outer space is not friendly to life. Extreme t...,NaN,NaN,NaN,NaN
1,sports,"Tennis, original name lawn tennis, game in whi...",NaN,NaN,NaN,NaN
2,business,One woman who frequently flew on Southwest was...,NaN,NaN,NaN,NaN
3,covid,"In December 2019, almost seven years after the...",NaN,NaN,NaN,NaN
4,science,Any life-forms that somehow find themselves in...,NaN,NaN,NaN,NaN


In [5]:
df.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4',
       'Unnamed: 5'],axis=1,inplace=True)                        #Removing the redundant columns
df.head()

,category,text
0,science,Outer space is not friendly to life. Extreme t...
1,sports,"Tennis, original name lawn tennis, game in whi..."
2,business,One woman who frequently flew on Southwest was...
3,covid,"In December 2019, almost seven years after the..."
4,science,Any life-forms that somehow find themselves in...


## Training Corpus

In [6]:
corpus_1=[]
for i in range(len(df)):
    corpus_1.append(df['text'][i])
corpus_1= ''.join(corpus_1)
corpus_1

"Outer space is not friendly to life. Extreme temperatures, low pressure and radiation can quickly degrade cell membranes and destroy DNA.Tennis, original name\xa0lawn tennis, game in which two opposing players (singles) or pairs of players (doubles) use tautly strung\xa0rackets\xa0to hit a\xa0ball\xa0of specified size, weight, and bounce over a\xa0net\xa0on a rectangular court.\xa0One woman who frequently flew on Southwest was constantly disappointed with every aspect of the company’s operation. In fact, she became known as the “Pen Pal” because after every flight she wrote in with a complaint.In December 2019, almost seven years after the MERS 2012 outbreak, a novel Coronavirus (2019-nCoV) surfaced in Wuhan in the Hubei region of China.Any life-forms that somehow find themselves in the void soon die.\xa0Unless they band together.As small communities, new research shows, some bacteria can withstand that harsh environment.Balls of\xa0Deinococcus\xa0bacteria as thin as five sheets of pa

In [7]:
stop_words = stopwords.words('english') #Using the stopwords of English Library

## Functions to preprocess the corpus

In [8]:
def remove_numbers(string):  #To remove numbers
    return ''.join([i for i in string if not i.isdigit()])

def remove_punctuations(string):  #To remove punctuations
    punctuation = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    no_punctuation = ""
    for char in string:
       if char not in punctuation:
           no_punctuation = no_punctuation + char
    return no_punctuation
def remove_stopwords(string):    #To remove the stopwords
    sentence=[]
    words=string.split()
    for word in words:
        if word not in stop_words:
            sentence.append(word)
    return ' '.join(sentence)
def stemmer(sentence):           #To do the stemming using Porter stemmer
    sent=[]
    ps = PorterStemmer()
    for i in sentence.split():
        trial=ps.stem(i)
        sent.append(trial)
    return ' '.join(sent)
            
def text_preprocessing(df):      #This function takes all the preprocessing functions and modifies the input corpus
    for i in range(len(df)):
        sen=df['text'][i]
        sen=remove_numbers(sen)
        sen=remove_punctuations(sen)
        sen=remove_stopwords(sen).lower()
        sen=stemmer(sen)
        df['text'][i]=sen

In [9]:
n_sci=df[df['category']=='science'].count()[0]  #number of sentences with category 'science'
n_spo=df[df['category']=='sports'].count()[0]    #number of sentences with category 'sports'
n_bus=df[df['category']=='business'].count()[0]   #number of sentences with category 'business'   
n_cov=df[df['category']=='covid'].count()[0]    #number of sentences with category 'covid'
total=n_sci+n_spo+n_cov+n_bus  #Total number of datapoints or sentences

## Prior Probabilities (P(Y)), Here Y are the respective categories

In [10]:
prior_pro_sci=n_sci/total  
prior_pro_spo=n_spo/total
prior_pro_bus=n_bus/total
prior_pro_cov=n_cov/total
print('Prior Probability of business:',prior_pro_bus)
print('Prior Probability of covid:',prior_pro_cov)
print('Prior Probability of science:',prior_pro_sci)
print('Prior Probability of sports:',prior_pro_spo)

Prior Probability of business: 0.2375
Prior Probability of covid: 0.2625
Prior Probability of science: 0.25
Prior Probability of sports: 0.25


In [11]:
text_preprocessing(df) #Dataframe after thw text pre processing of the sentences
df.head()

,category,text
0,science,outer space friendli life extrem temperatur lo...
1,sports,tenni origin name lawn tenni game two oppos pl...
2,business,one woman frequent flew southwest constantli d...
3,covid,in decemb almost seven year mer outbreak novel...
4,science,ani lifeform somehow find void soon die unless...


## Training Corpus after Text Preprocessing

In [12]:
docs = [row['text'] for index,row in df.iterrows()]
corpus_main=[]
for i in range(len(df)):
    corpus_main.append(df['text'][i])
corpus_main= ''.join(corpus_main)
corpus_main_vec=corpus_main.split()
main_vocab={}
for i in range(len(corpus_main_vec)):
    if corpus_main_vec[i] not in main_vocab:
        main_vocab[corpus_main_vec[i]]=1
    else:
        main_vocab[corpus_main_vec[i]]+=1
print(main_vocab)

{'outer': 5, 'space': 7, 'friendli': 1, 'life': 3, 'extrem': 3, 'temperatur': 1, 'low': 1, 'pressur': 2, 'radiat': 3, 'quickli': 1, 'degrad': 1, 'cell': 3, 'membran': 1, 'destroy': 1, 'dnatenni': 1, 'origin': 11, 'name': 2, 'lawn': 3, 'tenni': 13, 'game': 9, 'two': 3, 'oppos': 1, 'player': 7, 'singl': 2, 'pair': 1, 'doubl': 2, 'use': 6, 'tautli': 1, 'strung': 1, 'racket': 2, 'hit': 2, 'ball': 9, 'specifi': 2, 'size': 1, 'weight': 2, 'bounc': 1, 'net': 6, 'rectangular': 1, 'courtone': 1, 'woman': 1, 'frequent': 1, 'flew': 1, 'southwest': 3, 'constantli': 1, 'disappoint': 1, 'everi': 7, 'aspect': 1, 'company’': 1, 'oper': 1, 'in': 1, 'fact': 2, 'becam': 1, 'known': 5, '“pen': 1, 'pal”': 1, 'flight': 1, 'wrote': 1, 'complaintin': 1, 'decemb': 2, 'almost': 1, 'seven': 1, 'year': 5, 'mer': 4, 'outbreak': 5, 'novel': 1, 'coronaviru': 5, 'ncov': 2, 'surfac': 5, 'wuhan': 3, 'hubei': 2, 'region': 2, 'chinaani': 1, 'lifeform': 1, 'somehow': 2, 'find': 2, 'void': 1, 'soon': 1, 'die': 1, 'unless':

## Without using CountVectorizer (Getting Vocabulary and Class Conditional Probabilities)

Vocab of science corpus and P(X|Y=science)

In [13]:
sci_docs = [row['text'] for index,row in df.iterrows() if row['category'] == 'science'] 
corpus_sci=[]
for i in range(len(sci_docs)):
    corpus_sci.append(sci_docs[i])
corpus_sci= ' '.join(corpus_sci)
corpus_sci_vec=corpus_sci.split()
sci_vocab={}
for i in range(len(corpus_sci_vec)):
    if corpus_sci_vec[i] not in sci_vocab:
        sci_vocab[corpus_sci_vec[i]]=1
    else:
        sci_vocab[corpus_sci_vec[i]]+=1
probab_sci={}
Tot_sci=np.sum(list(sci_vocab.values()))
for i in range(len(corpus_sci_vec)):
    if corpus_sci_vec[i] not in probab_sci:
        probab_sci[corpus_sci_vec[i]]=sci_vocab[corpus_sci_vec[i]]/Tot_sci  #getting the conditional probabilities
#conditional Probability P(X|Y='sci') = P(X and Y)/P(Y) which means the probability of a word belonging 
#from the category science

#print(sci_vocab) #Prints the vectors along with the count
#print(probab_sci) #Prints the class conditional probability for science

Vocab of sports corpus and P(X|Y=sports)

In [14]:
spo_docs = [row['text'] for index,row in df.iterrows() if row['category'] == 'sports'] 
corpus_spo=[]
for i in range(len(spo_docs)):
    corpus_spo.append(spo_docs[i])
corpus_spo= ' '.join(corpus_spo)
corpus_spo_vec=corpus_spo.split()
spo_vocab={}
for i in range(len(corpus_spo_vec)):
    if corpus_spo_vec[i] not in spo_vocab:
        spo_vocab[corpus_spo_vec[i]]=1
    else:
        spo_vocab[corpus_spo_vec[i]]+=1
probab_spo={}
Tot_spo=np.sum(list(spo_vocab.values()))
for i in range(len(corpus_spo_vec)):
    if corpus_spo_vec[i] not in probab_spo:
        probab_spo[corpus_spo_vec[i]]=spo_vocab[corpus_spo_vec[i]]/Tot_spo    #getting the conditional probabilities
#conditional Probability P(X|Y='spo') = P(X and Y)/P(Y) which means the probability of a word belonging 
#from the category sports

#print(spo_vocab) #Prints the vectors along with the count
#print(probab_spo) #Prints the class conditional probability for sports

Vocab of business corpus and P(X|Y=business)

In [15]:
bus_docs = [row['text'] for index,row in df.iterrows() if row['category'] == 'business'] 
corpus_bus=[]
for i in range(len(bus_docs)):
    corpus_bus.append(bus_docs[i])
corpus_bus= ' '.join(corpus_bus)
corpus_bus_vec=corpus_bus.split()
bus_vocab={}
for i in range(len(corpus_bus_vec)):
    if corpus_bus_vec[i] not in bus_vocab:
        bus_vocab[corpus_bus_vec[i]]=1
    else:
        bus_vocab[corpus_bus_vec[i]]+=1

probab_bus={}
Tot_bus=np.sum(list(bus_vocab.values()))
for i in range(len(corpus_bus_vec)):
    if corpus_bus_vec[i] not in probab_bus:
        probab_bus[corpus_bus_vec[i]]=bus_vocab[corpus_bus_vec[i]]/Tot_bus    #getting the conditional probabilities
#conditional Probability P(X|Y='bus') = P(X and Y)/P(Y) which means the probability of a word belonging 
#from the category business

#print(bus_vocab) #Prints the vectors along with the count
#print(probab_bus) #Prints the class conditional probability for business

Vocab of covid corpus and P(X|Y=covid)

In [16]:
cov_docs = [row['text'] for index,row in df.iterrows() if row['category'] == 'covid'] 
corpus_cov=[]
for i in range(len(cov_docs)):
    corpus_cov.append(cov_docs[i])
corpus_cov= ' '.join(corpus_cov)
corpus_cov_vec=corpus_cov.split()
cov_vocab={}
for i in range(len(corpus_cov_vec)):
    if corpus_cov_vec[i] not in cov_vocab:
        cov_vocab[corpus_cov_vec[i]]=1
    else:
        cov_vocab[corpus_cov_vec[i]]+=1
probab_cov={}
Tot_cov=np.sum(list(cov_vocab.values()))
for i in range(len(corpus_cov_vec)):
    if corpus_cov_vec[i] not in probab_cov:
        probab_cov[corpus_cov_vec[i]]=cov_vocab[corpus_cov_vec[i]]/Tot_cov    #getting the conditional probabilities
#conditional Probability P(X|Y='cov') = P(X and Y)/P(Y) which means the probability of a word belonging 
#from the category covid

#print(cov_vocab) #Prints the vectors along with the count
#print(probab_cov) #Prints the class conditional probability for covid

## With CountVectorizer just to make TDM (Term Document Matrices)--Different Approach

## Term Document Matrices(tdm) of respective categories

In [17]:
sci_docs = [row['text'] for index,row in df.iterrows() if row['category'] == 'science']    #Corpus associated with 'science'
vec_sci = CountVectorizer()
X_sci = vec_sci.fit_transform(sci_docs)
tdm_sci = pd.DataFrame(X_sci.toarray(), columns=vec_sci.get_feature_names())

tdm_sci  #Term Document Matrix of category 'science'

,about,accident,add,affix,after,akihiko,also,ani,artifici,as,...,unprotect,void,wait,weight,well,withstand,work,worri,yamagishi,year
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,1,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0


In [18]:
spo_docs = [row['text'] for index,row in df.iterrows() if row['category'] == 'sports']    #Corpus associated with 'sports'

vec_spo = CountVectorizer()
X_spo = vec_spo.fit_transform(spo_docs)
tdm_spo = pd.DataFrame(X_spo.toarray(), columns=vec_spo.get_feature_names())

tdm_spo       #Term Document Matrix of category 'sports'

,accord,across,agre,allow,altern,although,amateur,ancient,angl,appeal,...,whenev,while,white,wide,winner,within,wool,world,worldwid,yellow
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,1,0,...,0,0,1,1,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [19]:
bus_docs = [row['text'] for index,row in df.iterrows() if row['category'] == 'business']   #Corpus associated with 'business'

vec_bus = CountVectorizer()
X_bus = vec_bus.fit_transform(bus_docs)
tdm_bus = pd.DataFrame(X_bus.toarray(), columns=vec_bus.get_feature_names())

tdm_bus      #Term Document Matrix for the category 'business'

,abandon,abus,also,alway,among,anyth,aren,argu,arriv,aspect,...,whose,woman,won,work,wors,would,wrong,wrote,you,yours
0,0,0,0,0,0,0,0,0,0,1,...,0,1,0,0,0,0,0,1,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
5,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,2,0
9,0,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
cov_docs = [row['text'] for index,row in df.iterrows() if row['category'] == 'covid'] #Corpus associated with 'covid'

vec_cov = CountVectorizer()
X_cov = vec_cov.fit_transform(cov_docs)
tdm_cov = pd.DataFrame(X_cov.toarray(), columns=vec_cov.get_feature_names())

tdm_cov      #Term Document Matrix of category 'covid'

,abil,ace,acut,ad,adapt,agent,allow,almost,also,andor,...,viru,virus,visibl,war,well,when,who,wuhan,year,zoonot
0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,1,0
1,0,0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,2,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
5,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
7,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


# Finding Class Conditional Probabilities P(X|Y), where X are the words and Y are the categories. It means what is the probability of a word being in a particular class

## Conditional Class Probability of Class 'science' i.e P(X|Y='science')

In [21]:
#Frequencies of words in the sentences of 'science' category
word_list_sci = vec_sci.get_feature_names();    
count_list_sci = X_sci.toarray().sum(axis=0) 
freq_sci = dict(zip(word_list_sci,count_list_sci)) 

#Probabilities of words in the sentences of 'science' category
prob_sci=[]
sum_sci=np.sum(count_list_sci)
for word,count in zip(word_list_sci,count_list_sci):
    prob_sci.append(count/sum_sci)
dict(zip(word_list_sci,prob_sci))       

{'about': 0.0045871559633027525,
 'accident': 0.0045871559633027525,
 'add': 0.0045871559633027525,
 'affix': 0.0045871559633027525,
 'after': 0.0045871559633027525,
 'akihiko': 0.0045871559633027525,
 'also': 0.009174311926605505,
 'ani': 0.0045871559633027525,
 'artifici': 0.0045871559633027525,
 'as': 0.0045871559633027525,
 'astrobiologist': 0.0045871559633027525,
 'astronaut': 0.009174311926605505,
 'back': 0.009174311926605505,
 'bacteria': 0.022935779816513763,
 'bacteriato': 0.0045871559633027525,
 'ball': 0.009174311926605505,
 'band': 0.0045871559633027525,
 'but': 0.009174311926605505,
 'calgari': 0.0045871559633027525,
 'canada': 0.0045871559633027525,
 'cell': 0.009174311926605505,
 'commun': 0.0045871559633027525,
 'could': 0.009174311926605505,
 'cramm': 0.009174311926605505,
 'dead': 0.009174311926605505,
 'degrad': 0.0045871559633027525,
 'deinococcu': 0.009174311926605505,
 'desicc': 0.0045871559633027525,
 'destroy': 0.0045871559633027525,
 'didn': 0.0045871559633027

## Conditional Class Probability of Class 'sports' i.e P(X|Y='sports')

In [22]:
#Frequencies of words in the sentences of 'sports' category
word_list_spo = vec_spo.get_feature_names();    
count_list_spo = X_spo.toarray().sum(axis=0) 
freq_spo = dict(zip(word_list_spo,count_list_spo))

#Probabilities of words in the sentences of 'sports' category
prob_spo=[]
sum_spo=np.sum(count_list_spo)
for word,count in zip(word_list_spo,count_list_spo):
    prob_spo.append(count/sum_spo)
dict(zip(word_list_spo,prob_spo))   

{'accord': 0.002288329519450801,
 'across': 0.002288329519450801,
 'agre': 0.002288329519450801,
 'allow': 0.002288329519450801,
 'altern': 0.002288329519450801,
 'although': 0.002288329519450801,
 'amateur': 0.002288329519450801,
 'ancient': 0.002288329519450801,
 'angl': 0.002288329519450801,
 'appeal': 0.002288329519450801,
 'asphalt': 0.002288329519450801,
 'associ': 0.002288329519450801,
 'attack': 0.002288329519450801,
 'attempt': 0.002288329519450801,
 'australia': 0.002288329519450801,
 'award': 0.002288329519450801,
 'back': 0.002288329519450801,
 'ball': 0.020594965675057208,
 'baselin': 0.002288329519450801,
 'began': 0.002288329519450801,
 'begin': 0.002288329519450801,
 'behind': 0.004576659038901602,
 'bodi': 0.002288329519450801,
 'bounc': 0.002288329519450801,
 'boundari': 0.002288329519450801,
 'britain': 0.004576659038901602,
 'broadcast': 0.002288329519450801,
 'broaden': 0.002288329519450801,
 'call': 0.009153318077803204,
 'case': 0.004576659038901602,
 'cement': 0

## Conditional Class Probability of Class 'business' i.e P(X|Y='business')

In [23]:
#Frequencies of words in the sentences of 'business' category
word_list_bus = vec_bus.get_feature_names();    
count_list_bus = X_bus.toarray().sum(axis=0) 
freq_bus = dict(zip(word_list_bus,count_list_bus))

#Probabilities of words in the sentences of 'business' category
prob_bus=[]
sum_bus=np.sum(count_list_bus)
for word,count in zip(word_list_bus,count_list_bus):
    prob_bus.append(count/sum_bus)
dict(zip(word_list_bus,prob_bus))  

{'abandon': 0.002702702702702703,
 'abus': 0.008108108108108109,
 'also': 0.005405405405405406,
 'alway': 0.016216216216216217,
 'among': 0.002702702702702703,
 'anyth': 0.002702702702702703,
 'aren': 0.002702702702702703,
 'argu': 0.002702702702702703,
 'arriv': 0.002702702702702703,
 'aspect': 0.002702702702702703,
 'assign': 0.002702702702702703,
 'attitud': 0.002702702702702703,
 'back': 0.005405405405405406,
 'bad': 0.010810810810810811,
 'balanc': 0.002702702702702703,
 'becam': 0.002702702702702703,
 'bethun': 0.005405405405405406,
 'better': 0.008108108108108109,
 'book': 0.002702702702702703,
 'bump': 0.002702702702702703,
 'busi': 0.010810810810810811,
 'but': 0.002702702702702703,
 'buy': 0.002702702702702703,
 'calcul': 0.002702702702702703,
 'can': 0.005405405405405406,
 'cancel': 0.002702702702702703,
 'caus': 0.005405405405405406,
 'ceo': 0.002702702702702703,
 'choic': 0.002702702702702703,
 'clear': 0.002702702702702703,
 'coin': 0.002702702702702703,
 'come': 0.002702

## Conditional Class Probability of Class 'covid' i.e P(X|Y='covid')

In [24]:
#Frequencies of words in the sentences of 'covid' category
word_list_cov = vec_cov.get_feature_names();    
count_list_cov = X_cov.toarray().sum(axis=0) 
freq_cov = dict(zip(word_list_cov,count_list_cov))

#Probabilities of words in the sentences of 'covid' category
prob_cov=[]
sum_cov=np.sum(count_list_cov)
for word,count in zip(word_list_cov,count_list_cov):
    prob_cov.append(count/sum_cov)
dict(zip(word_list_cov,prob_cov))  

{'abil': 0.003703703703703704,
 'ace': 0.003703703703703704,
 'acut': 0.011111111111111112,
 'ad': 0.003703703703703704,
 'adapt': 0.003703703703703704,
 'agent': 0.003703703703703704,
 'allow': 0.003703703703703704,
 'almost': 0.007407407407407408,
 'also': 0.003703703703703704,
 'andor': 0.003703703703703704,
 'angiotensinconvert': 0.003703703703703704,
 'anim': 0.007407407407407408,
 'antihiv': 0.003703703703703704,
 'antivir': 0.003703703703703704,
 'appear': 0.003703703703703704,
 'around': 0.003703703703703704,
 'attract': 0.003703703703703704,
 'avail': 0.003703703703703704,
 'bat': 0.003703703703703704,
 'both': 0.003703703703703704,
 'case': 0.003703703703703704,
 'caus': 0.018518518518518517,
 'cell': 0.003703703703703704,
 'centuri': 0.003703703703703704,
 'china': 0.018518518518518517,
 'citi': 0.003703703703703704,
 'coat': 0.003703703703703704,
 'cold': 0.007407407407407408,
 'common': 0.007407407407407408,
 'contain': 0.003703703703703704,
 'corona': 0.003703703703703704

In [25]:
#Corpus 
docs = [row['text'] for index,row in df.iterrows()]

#Vectorizing the feature words of corpus
vec = CountVectorizer()
X = vec.fit_transform(docs)

#To Calculate total number of feature words
total_features = len(vec.get_feature_names())
total_features   

709

In [26]:
total_cnts_features_sci = count_list_sci.sum(axis=0)  #Total feature counts of science corpus 
total_cnts_features_spo = count_list_spo.sum(axis=0)  #Total feature counts of sports corpus 
total_cnts_features_bus = count_list_bus.sum(axis=0)  #Total feature counts of business corpus 
total_cnts_features_cov = count_list_cov.sum(axis=0)  #Total feature counts of covid corpus 

In [27]:
print('Features in science:',total_cnts_features_sci,'Features in science:',total_cnts_features_spo,'Features in science:',total_cnts_features_bus,'Features in science:',total_cnts_features_cov)

Features in science: 218 Features in science: 437 Features in science: 370 Features in science: 270


In [28]:
# Function to preprocess the sentences of test set, one by one, and return the tokenized form of it
def new_word_list(sentence):
    new_sentence = [sentence]
    new_sentence=remove_numbers(new_sentence)
    new_sentence=remove_punctuations(new_sentence)
    new_sentence=remove_stopwords(new_sentence)
    new_sentence=stemmer(new_sentence)
    new_word_list = word_tokenize(new_sentence)
    return new_word_list

## Functions to find Posterior Probability Distribution of each class for a tokenized sentence i.e P(Y|X), Here, to avoid the cases of zero probabilities, I have used "LAPLACE SMOOTHING" 

## The posterior distributions of the respective classes for each statement is printed below. To look at the posterior probabilities of tokens, just unhash the print statements inside the functions below. I avoided printing it, because it was taking up too much space.

In [29]:
# probability of the sentence to be of class 'science'
def posterior_distribution_sci(new_word_list,freq_sci,count,total_cnts_features_sci,total_features):
    prob_sci_with_ls = []
    for word in new_word_list:
        if word in freq_sci.keys():
            count = freq_sci[word]
        else:
            count = 0
        prob_sci_with_ls.append((count + 1)/(total_cnts_features_sci + total_features)) #Laplace Smoothing
    #print('Posterior probabilities of tokenized words in the sentence for class science:', dict(zip(new_word_list,prob_sci_with_ls)),'\n')
    print('posterior probability for science of tokenized statement--', ' '.join(new_word_list),'--is:',np.prod(prob_sci_with_ls),'\n')
    return np.prod(prob_sci_with_ls)

# probability of the sentence to be of class 'sports'
def posterior_distribution_spo(new_word_list,freq_spo,count,total_cnts_features_spo,total_features):
    prob_spo_with_ls = []
    for word in new_word_list:
        if word in freq_spo.keys():
            count = freq_spo[word]
        else:
            count = 0
        prob_spo_with_ls.append((count + 1)/(total_cnts_features_spo + total_features))  #Laplace Smoothing
    #print('Posterior probabilities of tokenized words in the sentence for class sports:', dict(zip(new_word_list,prob_sci_with_ls)),'\n')
    print('posterior probability for sports of tokenized statement--', ' '.join(new_word_list),'--is:',np.prod(prob_spo_with_ls),'\n')
    return np.prod(prob_spo_with_ls)

# probability of the sentence to be of class 'business'
def posterior_distribution_bus(new_word_list,freq_bus,count,total_cnts_features_bus,total_features):
    prob_bus_with_ls = []
    for word in new_word_list:
        if word in freq_bus.keys():
            count = freq_bus[word]
        else:
            count = 0
        prob_bus_with_ls.append((count + 1)/(total_cnts_features_bus + total_features))  #Laplace Smoothing
    #print('Posterior probabilities of tokenized words in the sentence for class business:', dict(zip(new_word_list,prob_sci_with_ls)),'\n')
    print('posterior probability for business of tokenized statement--', ' '.join(new_word_list),'--is:',np.prod(prob_bus_with_ls),'\n')
    return np.prod(prob_bus_with_ls)

# probability of the sentence to be of class 'covid'
def posterior_distribution_cov(new_word_list,freq_cov,count,total_cnts_features_cov,total_features):
    prob_cov_with_ls = []
    for word in new_word_list:
        if word in freq_cov.keys():
            count = freq_cov[word]
        else:
            count = 0
        prob_cov_with_ls.append((count + 1)/(total_cnts_features_cov + total_features))   #Laplace Smoothing
    #print('Posterior probabilities of tokenized words in the sentence for class covid:', dict(zip(new_word_list,prob_sci_with_ls)),'\n')
    print('posterior probability for covid of tokenized statement--', ' '.join(new_word_list),'--is:',np.prod(prob_cov_with_ls),'\n')
    return np.prod(prob_cov_with_ls)

## Testing on the 'testdata' corpus
Notice: The test dataset and train dataset were being stemmed and so because of that I got weird stem words, anyways the benefit of stemming here is that it'd work well in any variation of a particular stem word.

In [30]:
df_test=pd.read_csv('testdata.csv')

In [57]:
#Finding the probabilities of the sentence being in a particular class and then assigning the class with maximum probabilty

preds=[]  #preds will contain the predicted class 
for i in range(len(df_test)):
    new_word_list1=new_word_list(df_test['text'][i])
    cov=posterior_distribution_cov(new_word_list1,freq_cov,count,total_cnts_features_cov,total_features)
    bus=posterior_distribution_bus(new_word_list1,freq_bus,count,total_cnts_features_bus,total_features)
    spo=posterior_distribution_spo(new_word_list1,freq_spo,count,total_cnts_features_spo,total_features)
    sci=posterior_distribution_sci(new_word_list1,freq_sci,count,total_cnts_features_sci,total_features)
    maxi=max(max(cov,bus),max(spo,sci))
    if maxi==cov:
        preds.append('covid')
    elif maxi==bus:
        preds.append('business')
    elif maxi==spo:
        preds.append('sports')
    else:
        preds.append('science')

posterior probability for covid of tokenized statement-- He estim 1000micromet pellet could surviv eight year float space --is: 4.9457604044351375e-30 

posterior probability for business of tokenized statement-- He estim 1000micromet pellet could surviv eight year float space --is: 4.675042390723925e-31 

posterior probability for sports of tokenized statement-- He estim 1000micromet pellet could surviv eight year float space --is: 2.5594922783919217e-31 

posterior probability for science of tokenized statement-- He estim 1000micromet pellet could surviv eight year float space --is: 1.0140960486616668e-26 

posterior probability for covid of tokenized statement-- “ that ’ enough time potenti get mars ” say rare meteor might even abl travel mar earth month year --is: 1.2230272989039206e-59 

posterior probability for business of tokenized statement-- “ that ’ enough time potenti get mars ” say rare meteor might even abl travel mar earth month year --is: 3.1472670751294558e-59 

poster

## Accuracy Score for testing set

In [58]:
accuracy=np.sum(preds==df_test['category'])/len(preds)
print('The accuracy score of the above Naive Bayes is:', accuracy)

The accuracy score of the above Naive Bayes is: 0.85


# Problem Statement-2

In [33]:
data=pd.read_csv('40.csv')    #reading dataset

In [34]:
data.head()

,text
0,"In the midst of the COVID-19 pandemic, eating ..."
1,While there are no specific foods that can hel...
2,You may not be able to share meals with friend...
3,Eating a healthy diet is not about strict limi...
4,Healthy eating doesn’t have to be overly compl...


In [35]:
#Getting the Training Corpus
corpus=[]
for i in range(len(data)):
    corpus.append(data['text'][i])
corpus= '. '.join(corpus)
corpus

'In the midst of the COVID-19 pandemic, eating healthy food remains an important part of maintaining your health.. While there are no specific foods that can help protect you from the virus, a nutritious diet can boost your immune system or help you fight off symptoms.. You may not be able to share meals with friends and loved ones, but there are lots of other ways to eat well and support your health at this difficult time.. Eating a healthy diet is not about strict limitations, staying unrealistically thin, or depriving yourself of the foods you love. Rather, it’s about feeling great, having more energy, improving your health, and boosting your mood.. Healthy eating doesn’t have to be overly complicated. If you feel overwhelmed by all the conflicting nutrition and diet advice out there, you’re not alone.. we all need a balance of protein, fat, carbohydrates, fiber, vitamins, and minerals in our diets to sustain a healthy body.\xa0. Protein\xa0gives you the energy to get up and go—and 

## Training Corpus after Text Pre-Processing

In [36]:
#Text Pre-Processing of te corpus
corpus=remove_numbers(corpus)
corpus=remove_punctuations(corpus)
corpus=remove_stopwords(corpus)
#corpus=stemmer(corpus) #Not doing stemming as it is not important here
corpus  #corpus after the pre-processing

'In midst COVID pandemic eating healthy food remains important part maintaining health While specific foods help protect virus nutritious diet boost immune system help fight symptoms You may able share meals friends loved ones lots ways eat well support health difficult time Eating healthy diet strict limitations staying unrealistically thin depriving foods love Rather it’s feeling great energy improving health boosting mood Healthy eating doesn’t overly complicated If feel overwhelmed conflicting nutrition diet advice you’re alone need balance protein fat carbohydrates fiber vitamins minerals diets sustain healthy body Protein gives energy get go—and keep going—while also supporting mood cognitive function Too much protein harmful people kidney disease latest research suggests many us need highquality protein especially age While bad fats wreck diet increase risk certain diseases good fats protect brain heart In fact healthy fats—such omegas—are vital physical emotional health Includi

In [37]:
#Tokenization of the corpus 
tokens=corpus.split(' ')

In [38]:
#Counting the frequency of tokens
count={}
for token in tokens:
    if token not in count.keys():
        count[token]=1
    else:
        count[token]+=1

In [39]:
total=np.sum(list(count.values())) #total tokens taking into account their frequency as well

## Prior Probability Distribution(P(Y)), Here Y are the tokenized words

In [40]:
#Prior Probability Distribution 
prob={}
for token in np.unique(tokens):
    prob[token]=count[token]/total
prob

{'As': 0.0033333333333333335,
 'Brussels': 0.0016666666666666668,
 'But': 0.0033333333333333335,
 'COVID': 0.0016666666666666668,
 'Carbohydrates': 0.0016666666666666668,
 'Control': 0.0016666666666666668,
 'Cook': 0.0016666666666666668,
 'Cooking': 0.0016666666666666668,
 'Cutting': 0.0016666666666666668,
 'D': 0.0016666666666666668,
 'Drink': 0.0016666666666666668,
 'Eating': 0.006666666666666667,
 'Focus': 0.0016666666666666668,
 'For': 0.0016666666666666668,
 'Fruit': 0.0016666666666666668,
 'Healthy': 0.0016666666666666668,
 'If': 0.0016666666666666668,
 'In': 0.005,
 'Including': 0.0016666666666666668,
 'Instead': 0.0016666666666666668,
 'It': 0.0016666666666666668,
 'It’s': 0.0033333333333333335,
 'K': 0.0016666666666666668,
 'Many': 0.0016666666666666668,
 'Naturally': 0.0016666666666666668,
 'Protein': 0.0016666666666666668,
 'Rather': 0.0016666666666666668,
 'Replacing': 0.0016666666666666668,
 'Satisfy': 0.0016666666666666668,
 'Switching': 0.0016666666666666668,
 'TV': 0.00

## Using Bigrams Model and Markov Chain, where it is assumed that the present state is represented by the last few states(in case of Bigrams, it is the states 't' and 't-1', respectively)

In [41]:
bigrams=[(tokens[i],tokens[i+1]) for i in range(0,len(tokens)-1)] #pairing the words and getting the bigrams tuples list
bigrams

[('In', 'midst'),
 ('midst', 'COVID'),
 ('COVID', 'pandemic'),
 ('pandemic', 'eating'),
 ('eating', 'healthy'),
 ('healthy', 'food'),
 ('food', 'remains'),
 ('remains', 'important'),
 ('important', 'part'),
 ('part', 'maintaining'),
 ('maintaining', 'health'),
 ('health', 'While'),
 ('While', 'specific'),
 ('specific', 'foods'),
 ('foods', 'help'),
 ('help', 'protect'),
 ('protect', 'virus'),
 ('virus', 'nutritious'),
 ('nutritious', 'diet'),
 ('diet', 'boost'),
 ('boost', 'immune'),
 ('immune', 'system'),
 ('system', 'help'),
 ('help', 'fight'),
 ('fight', 'symptoms'),
 ('symptoms', 'You'),
 ('You', 'may'),
 ('may', 'able'),
 ('able', 'share'),
 ('share', 'meals'),
 ('meals', 'friends'),
 ('friends', 'loved'),
 ('loved', 'ones'),
 ('ones', 'lots'),
 ('lots', 'ways'),
 ('ways', 'eat'),
 ('eat', 'well'),
 ('well', 'support'),
 ('support', 'health'),
 ('health', 'difficult'),
 ('difficult', 'time'),
 ('time', 'Eating'),
 ('Eating', 'healthy'),
 ('healthy', 'diet'),
 ('diet', 'strict'),
 

In [42]:
#To get the counts of the respective bigrams tokens that were obtained above
countb={}
for token in bigrams:
    if token not in countb.keys():
        countb[token]=1
    else:
        countb[token]+=1
countb

{('In', 'midst'): 1,
 ('midst', 'COVID'): 1,
 ('COVID', 'pandemic'): 1,
 ('pandemic', 'eating'): 1,
 ('eating', 'healthy'): 1,
 ('healthy', 'food'): 1,
 ('food', 'remains'): 1,
 ('remains', 'important'): 1,
 ('important', 'part'): 1,
 ('part', 'maintaining'): 1,
 ('maintaining', 'health'): 1,
 ('health', 'While'): 1,
 ('While', 'specific'): 1,
 ('specific', 'foods'): 1,
 ('foods', 'help'): 1,
 ('help', 'protect'): 1,
 ('protect', 'virus'): 1,
 ('virus', 'nutritious'): 1,
 ('nutritious', 'diet'): 1,
 ('diet', 'boost'): 1,
 ('boost', 'immune'): 1,
 ('immune', 'system'): 1,
 ('system', 'help'): 1,
 ('help', 'fight'): 1,
 ('fight', 'symptoms'): 1,
 ('symptoms', 'You'): 1,
 ('You', 'may'): 1,
 ('may', 'able'): 1,
 ('able', 'share'): 1,
 ('share', 'meals'): 1,
 ('meals', 'friends'): 1,
 ('friends', 'loved'): 1,
 ('loved', 'ones'): 1,
 ('ones', 'lots'): 1,
 ('lots', 'ways'): 1,
 ('ways', 'eat'): 1,
 ('eat', 'well'): 1,
 ('well', 'support'): 1,
 ('support', 'health'): 1,
 ('health', 'difficult

## Class Conditional Probability(P(X|Y)), i.e let's say words ('w1', 'w2') be the bigram tuple, so this is the probability of w1(X) coming before the w2(Y)

Since P(X|Y) = P(X and Y)/P(Y), here 'w1' is X and 'w2' is Y

In [43]:
#class conditional Probabilities -- P(X|Y)
cl_con={}
for i in range(len(bigrams)):
    cl_con[bigrams[i]]=countb[bigrams[i]]/count[bigrams[i][1]] #P(W1 and W2)/P(W2)=Count(w1 w2)/count(w2)
cl_con 

{('In', 'midst'): 1.0,
 ('midst', 'COVID'): 1.0,
 ('COVID', 'pandemic'): 1.0,
 ('pandemic', 'eating'): 0.125,
 ('eating', 'healthy'): 0.07692307692307693,
 ('healthy', 'food'): 0.1,
 ('food', 'remains'): 1.0,
 ('remains', 'important'): 0.3333333333333333,
 ('important', 'part'): 1.0,
 ('part', 'maintaining'): 1.0,
 ('maintaining', 'health'): 0.2,
 ('health', 'While'): 0.3333333333333333,
 ('While', 'specific'): 1.0,
 ('specific', 'foods'): 0.1111111111111111,
 ('foods', 'help'): 0.1111111111111111,
 ('help', 'protect'): 0.5,
 ('protect', 'virus'): 1.0,
 ('virus', 'nutritious'): 1.0,
 ('nutritious', 'diet'): 0.09090909090909091,
 ('diet', 'boost'): 0.5,
 ('boost', 'immune'): 1.0,
 ('immune', 'system'): 1.0,
 ('system', 'help'): 0.1111111111111111,
 ('help', 'fight'): 1.0,
 ('fight', 'symptoms'): 0.5,
 ('symptoms', 'You'): 0.3333333333333333,
 ('You', 'may'): 1.0,
 ('may', 'able'): 1.0,
 ('able', 'share'): 1.0,
 ('share', 'meals'): 0.3333333333333333,
 ('meals', 'friends'): 1.0,
 ('frien

In [44]:
#Probability of occurence of certain bigram in the set of all bigrams
probb={}
totalb=np.sum(list(countb.values()))
for token in bigrams:
    probb[token]=countb[token]/totalb
probb

{('In', 'midst'): 0.001669449081803005,
 ('midst', 'COVID'): 0.001669449081803005,
 ('COVID', 'pandemic'): 0.001669449081803005,
 ('pandemic', 'eating'): 0.001669449081803005,
 ('eating', 'healthy'): 0.001669449081803005,
 ('healthy', 'food'): 0.001669449081803005,
 ('food', 'remains'): 0.001669449081803005,
 ('remains', 'important'): 0.001669449081803005,
 ('important', 'part'): 0.001669449081803005,
 ('part', 'maintaining'): 0.001669449081803005,
 ('maintaining', 'health'): 0.001669449081803005,
 ('health', 'While'): 0.001669449081803005,
 ('While', 'specific'): 0.001669449081803005,
 ('specific', 'foods'): 0.001669449081803005,
 ('foods', 'help'): 0.001669449081803005,
 ('help', 'protect'): 0.001669449081803005,
 ('protect', 'virus'): 0.001669449081803005,
 ('virus', 'nutritious'): 0.001669449081803005,
 ('nutritious', 'diet'): 0.001669449081803005,
 ('diet', 'boost'): 0.001669449081803005,
 ('boost', 'immune'): 0.001669449081803005,
 ('immune', 'system'): 0.001669449081803005,
 ('s

## Predictions of the last words of sentences in 10.csv

In [45]:
data_test=pd.read_csv('10.csv') #Dataset_used

In [46]:
#To get the tokenized form of the sentences given in the dataset and storing them in b which is a list of lists
b=[0]*len(data_test)
for i in range(len(data_test)):
    data_test['text'][i]=remove_punctuations(data_test['text'][i])
    data_test['text'][i]=remove_stopwords(data_test['text'][i])
    b[i]=data_test['text'][i].split(' ')
b

[['Eating', 'healthy', 'food', 'important', 'maintainng', 'good'],
 ['Following', 'healthy', 'diet', 'boost'],
 ['Avoid',
  'eating',
  'chemical',
  'additives',
  'added',
  'sugars',
  'diet',
  'Switch',
  'healthy'],
 ['Your', 'diet', 'rich', 'vitamins', 'DK', 'calcium'],
 ['The', 'healthier', 'food', 'eat', 'better', 'you’ll', 'feel'],
 ['Dehydration',
  'causes',
  'tiredness',
  'low',
  'energy',
  'head',
  'aches',
  'Drink',
  'plenty'],
 ['People', 'kidney', 'disease', 'avoid', 'eating', 'high', 'amounts'],
 ['We',
  'avoid',
  'eating',
  'transfats',
  'Eating',
  'healthy',
  'fats',
  'dietary',
  'fibre',
  'help',
  'us',
  'lose'],
 ['Mindless',
  'eating',
  'often',
  'caused',
  'eating',
  'alone',
  'We',
  'avoid',
  'eating',
  'front',
  'TV'],
 ['Eating', 'junk', 'food', 'make', 'feel', 'uncomfortable']]

In [47]:
#res dictionary is used to store the bigrams tuple along with their probability of occurence among all bigram tuples
#res is used for later reference
res={}
k=list(probb.keys())
for i in range(len(b)):
    for j in range(len(k)):
        if b[i][-1] == k[j][0]:
            res[k[j]]=probb[k[j]]

r1={}
r2={}
keyss=list(res.keys())
for i in range(len(keyss)):
    r1[keyss[i][0]]=prob[keyss[i][0]] #Not Important
    r2[keyss[i][1]]=prob[keyss[i][1]] #Not Important
keyss

[('good', 'fats'),
 ('boost', 'immune'),
 ('boost', 'wellbeing'),
 ('healthy', 'food'),
 ('healthy', 'diet'),
 ('healthy', 'body'),
 ('healthy', 'fats—such'),
 ('healthy', 'fat'),
 ('healthy', 'choices'),
 ('healthy', 'alternatives'),
 ('healthy', 'fats'),
 ('healthy', 'The'),
 ('healthy', 'lunch'),
 ('healthy', 'sides'),
 ('calcium', 'diet'),
 ('calcium', 'get'),
 ('calcium', 'job'),
 ('feel', 'overwhelmed'),
 ('feel', 'meal'),
 ('feel', 'uncomfortable'),
 ('feel', 'satisfied'),
 ('plenty', 'water'),
 ('plenty', 'ways'),
 ('amounts', 'sugar'),
 ('lose', 'weight'),
 ('TV', 'computer'),
 ('uncomfortable', 'nauseous')]

## Posterior Probabilities (P(Y|X)), i.e. probability of the word being the predicted word, after the last word of the sentence, since Markov state 't-1' is only important under the assumption of bigram model.
## let's say words ('w1', 'w2') be the bigram tuple, so this is the probability of w2(Y) coming after the w1(X)

Since P(Y|X) = P(X and Y)/P(X), here 'w1' is X and 'w2' is Y

In [48]:
#Posterior Probability distribution of the test dataset sentences, with first element of each tuple being the last 
#word of each of the sentences and second element being the words in the bigram set which are associated with the first
#word
div={}
for i in range(len(keyss)):
    div[keyss[i]]=countb[keyss[i]]/count[keyss[i][0]] #Count(w1 w2)/count(w1)
div #contains the (last_word_of_each_sentence, possible_word_coming_after_the_first_tuple_word_in_the_training_corpus), along with their probabilities

{('good', 'fats'): 1.0,
 ('boost', 'immune'): 0.5,
 ('boost', 'wellbeing'): 0.5,
 ('healthy', 'food'): 0.07692307692307693,
 ('healthy', 'diet'): 0.23076923076923078,
 ('healthy', 'body'): 0.07692307692307693,
 ('healthy', 'fats—such'): 0.07692307692307693,
 ('healthy', 'fat'): 0.07692307692307693,
 ('healthy', 'choices'): 0.07692307692307693,
 ('healthy', 'alternatives'): 0.07692307692307693,
 ('healthy', 'fats'): 0.07692307692307693,
 ('healthy', 'The'): 0.07692307692307693,
 ('healthy', 'lunch'): 0.07692307692307693,
 ('healthy', 'sides'): 0.07692307692307693,
 ('calcium', 'diet'): 0.3333333333333333,
 ('calcium', 'get'): 0.3333333333333333,
 ('calcium', 'job'): 0.3333333333333333,
 ('feel', 'overwhelmed'): 0.25,
 ('feel', 'meal'): 0.25,
 ('feel', 'uncomfortable'): 0.25,
 ('feel', 'satisfied'): 0.25,
 ('plenty', 'water'): 0.5,
 ('plenty', 'ways'): 0.5,
 ('amounts', 'sugar'): 1.0,
 ('lose', 'weight'): 1.0,
 ('TV', 'computer'): 1.0,
 ('uncomfortable', 'nauseous'): 1.0}

In [49]:
#For each bigram tupple in the posterior probabilities above, there are certain cases where the first elements aren't unique
#So for prediction of the most probable second element, we took the one with maximum posterior probability
#saved the pair in dictionary tuplee
tuplee={}
for key,value in div.items():
    if key[0] not in tuplee:
        tuplee[key[0]]=key
    else:
        if div[tuplee[key[0]]] < value:
            tuplee[key[0]]=key
tuplee       #format: 'first_tuple_word':('first_tuple_word','predicted_tupke_word_with_maximum_posterior_probability') 

{'good': ('good', 'fats'),
 'boost': ('boost', 'immune'),
 'healthy': ('healthy', 'diet'),
 'calcium': ('calcium', 'diet'),
 'feel': ('feel', 'overwhelmed'),
 'plenty': ('plenty', 'water'),
 'amounts': ('amounts', 'sugar'),
 'lose': ('lose', 'weight'),
 'TV': ('TV', 'computer'),
 'uncomfortable': ('uncomfortable', 'nauseous')}

In [50]:
#Getting the first elements in div tuple
first_tuple_elements = [a_tuple[0] for a_tuple in list(div.keys())]
first_tuple_elements

['good',
 'boost',
 'boost',
 'healthy',
 'healthy',
 'healthy',
 'healthy',
 'healthy',
 'healthy',
 'healthy',
 'healthy',
 'healthy',
 'healthy',
 'healthy',
 'calcium',
 'calcium',
 'calcium',
 'feel',
 'feel',
 'feel',
 'feel',
 'plenty',
 'plenty',
 'amounts',
 'lose',
 'TV',
 'uncomfortable']

In [51]:
#result dictionary contains the predicted word and last word of each sentences along with their posterior probabilities
result={}
for key in list(tuplee.values()):
    result[key]=div[key]
result

{('good', 'fats'): 1.0,
 ('boost', 'immune'): 0.5,
 ('healthy', 'diet'): 0.23076923076923078,
 ('calcium', 'diet'): 0.3333333333333333,
 ('feel', 'overwhelmed'): 0.25,
 ('plenty', 'water'): 0.5,
 ('amounts', 'sugar'): 1.0,
 ('lose', 'weight'): 1.0,
 ('TV', 'computer'): 1.0,
 ('uncomfortable', 'nauseous'): 1.0}

## Predicted last word after concatenating it with the sentences of 10.csv dataset

In [52]:
data_tests=pd.read_csv('10.csv')
for i in range(len(data_tests)):
    data_tests['text'][i]=remove_punctuations(data_tests['text'][i])
preds=[]
for i in range(len(data_tests)):
    preds.append(data_tests['text'][i]+list(result.keys())[i][1])
preds

['Eating healthy food is important for maintainng good fats',
 'Following a healthy diet will boost your immune',
 'Avoid eating chemical additives added sugars in your diet Switch to a healthy diet',
 'Your diet should be rich of vitamins DK calcium and diet',
 'The healthier the food you eat the better you’ll feel after a overwhelmed',
 'Dehydration causes tiredness low energy and head aches Drink plenty of water',
 'People with kidney disease should avoid eating high amounts of sugar',
 'We should avoid eating transfats Eating healthy fats and dietary fibre can help us lose weight',
 'Mindless eating is often caused by eating alone We should avoid eating while we are in front of a TV or a computer',
 'Eating more junk food will make you feel uncomfortable and nauseous']

## Inferences from the predictions: Apart from sentences 4(Predicted label: 'diet') and 5(Predicted label:'overwhelmed'), our model seems to perform well on other test cases, although sentence 4 isn't a redundant prediction because word 'diet' goes well along the other words in the sentence unlike the case with 'overwhelmed' in sentence 5. Also, the predictions are heavily dependant on the training corpus as well, so that is other factor to consider while modeling 